<a href="https://colab.research.google.com/github/Neuralwood-Net/face-recognizer-9000/blob/main/notebooks/pretraining_celeba_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Celeb A Neural Net Test

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Nov 11 13:05:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 31.6 gigabytes of available RAM

You are using a high-RAM runtime!


### Imports

In [3]:
import time
import os
import copy

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms

from google.cloud import storage

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Read and prepare the data

In [ ]:
from google.cloud import storage


client = storage.Client()

bucket_name = "tdt4173-datasets"
bucket = client.get_bucket(bucket_name)

zipfilename = "/content/celeba/img_align_celeba.zip"
blob_name = "faces/augmented_images/augmented_images_final_226880.zip"
blob = bucket.get_blob(blob_name)

blob.download_to_filename(zipfilename)

In [13]:
data_transforms = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),                                  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    # transforms.Normalize([0.5], [0.5]),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

image_datasets = {x: datasets.CelebA(
    root="/home/jupyter/data",
    split=x,
    target_type="identity",
    transform=data_transforms,
    download=True,
) for x in ["train", "valid"]}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].identity
NUM_CLASSES = 10_177

Files already downloaded and verified
Files already downloaded and verified


In [14]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    # mean = np.array([0.5])
    # std = np.array([0.5])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

print(inputs.shape)

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torchvision/datasets/celeba.py", line 141, in __getitem__
    X = self.transform(X)
  File "/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 70, in __call__
    img = t(img)
  File "/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 175, in __call__
    return F.normalize(tensor, self.mean, self.std, self.

### Create functions for training, validation, and evaluation

In [ ]:
def validate(model, val_loader, loss_func):
    correct = 0
    for batch_idx, (X_batch, y_batch) in enumerate(val_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        output = model(X_batch.float())
        loss = loss_func(output, y_batch)

        predicted = torch.argmax(output, 1)
        correct += (predicted == y_batch).sum()

    acc = float(correct*100) / float(BATCH_SIZE*(batch_idx+1))

    return loss, acc

In [ ]:
def fit(model, train_loader, val_loader):
    optimizer = torch.optim.Adam(model.parameters())
    loss_function = nn.CrossEntropyLoss()
    EPOCHS = 20
    EVAL_EVERY = 50
    model.train()

    for epoch in range(1, EPOCHS + 1):
        correct = 0
        it = tqdm(train_loader, desc=f"Epoch: {epoch}")
        for batch_idx, (X_batch, y_batch) in enumerate(it):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch.float())
            loss = loss_function(output, y_batch)
            loss.backward()
            optimizer.step()

            predicted = torch.argmax(output, 1)
            correct += (predicted == y_batch).sum()

            if batch_idx % EVAL_EVERY == 0:
                acc = float(correct*100) / float(BATCH_SIZE*(batch_idx+1))

                val_loss, val_acc = validate(model, val_loader, loss_function)


                it.set_postfix({
                    "Loss": f"{loss.item():.4f}",
                    "Acc": f"{acc:.2f}",
                    "Val loss": f"{val_loss.item():.4f}",
                    "Val acc": f"{val_acc:.2f}",
                })

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0

    for test_imgs, test_labels in test_loader:
        test_imgs, test_labels = test_imgs.to(device), test_labels.to(device)
        output = model(test_imgs.float())
        predicted = torch.argmax(output, dim=1)
        correct += (predicted == test_labels).sum()

    print(f"Test accuracy: {float(correct) / (len(test_loader)*BATCH_SIZE):.3f}%")

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            it = tqdm(dataloaders[phase], desc=f"Epoch: {epoch} ({phase})")
            for inputs, labels in it:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                it.set_postfix({"Loss": loss.item()})
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Prepare and train the CNN

In [8]:
class CNN(nn.Module):
    size_after_conv = 8 * 8 * 256
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(   
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            # nn.Dropout(),
            nn.Conv2d(32, 32, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv2d(128, 256, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.classify = nn.Sequential(
            nn.Linear(self.size_after_conv, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, NUM_CLASSES),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.size_after_conv)
        x = self.classify(x)

        return F.log_softmax(x, dim=1)

In [9]:
cnn = CNN().to(device)
print(cnn)

CNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): ReLU()
    (9): Dropout(p=0.5, inplace=False)
  )
  (classify): Sequential(
    (0): Linear(in_features=65536, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=10177, bias=True)
  )
)


In [10]:
optimizer = torch.optim.Adam(cnn.parameters())
loss_function = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

train_model(cnn, loss_function, optimizer, exp_lr_scheduler, num_epochs=25)

Epoch: 0 (train):   0%|          | 0/40693 [00:00<?, ?it/s]

Epoch 0/24
----------


Epoch: 0 (train):   0%|          | 0/40693 [00:00<?, ?it/s]


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torchvision/datasets/celeba.py", line 141, in __getitem__
    X = self.transform(X)
  File "/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 70, in __call__
    img = t(img)
  File "/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 1174, in __call__
    return F.to_grayscale(img, num_output_channels=self.

In [ ]:
train_loader, val_loader = dataloaders["train"], dataloaders["val"]
fit(cnn, train_loader, val_loader)


Epoch: 1:   0%|          | 0/5798 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
evaluate(cnn, test_loader)

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
images, labels = next(iter(test_loader))

for idx, (image, label) in enumerate(zip(images, labels)):
    pred = int(torch.argmax(cnn(image.view(-1, 1, 64, 64).to(device))))
    convert = {0: "Lars", 1: "Morgan", 2: "Kjartan", 3: "Ingen"}
    
    plt.imshow(image.view(64, 64).cpu(), cmap="gray")
    plt.text(2, 54, f"Image {idx + 1}", fontsize=14, color="white")
    plt.text(2, 58, f"Predicted: `{convert[pred]}`", fontsize=14, color="white")
    plt.text(2, 62, f"Actual   : `{convert[label.item()]}`", fontsize=14, color="white")
    plt.pause(0.05)

In [ ]:
import cv2

In [ ]:
cnn.eval()

class Label:
    def __init__(self, label):
        self.label = label
    
    def item(self):
        return self.label


filename = "/content/lars_5.png"
image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (64, 64)) / 255.0
plt.imshow(image, cmap="gray")
convert = {0: "Lars", 1: "Morgan", 2: "Kjartan", 3: "Ingen"}

pred = int(torch.argmax(cnn(torch.Tensor(image).view(-1, 1, 64, 64).to(device))))
# plt.text(2, 58, f"Predicted: `{convert[pred]}`", fontsize=14, color="white")
# plt.text(2, 62, f"Actual   : `{convert[label.item()]}`", fontsize=14, color="white")

if "morgan" in filename:
    label = Label(1)
elif "lars" in filename:
    label = Label(0)
else:
    label = Label(3)

print(f"Predicted: `{convert[pred]}`" )
print(f"Actual   : `{convert[label.item()]}`")

# plt.imshow(image.view(64, 64).cpu().to_numpy(), cmap="gray")
